In [ ]:
import logging
from datetime import datetime

current_file_name = "16_Evaluation"

dt_string = datetime.now().strftime("%Y%m%d_%H%M%S")
log_file = f"logs/{current_file_name}/{dt_string}.log"
logging.basicConfig(level=logging.INFO, filename=log_file,filemode="w", format="%(asctime)s %(levelname)s %(message)s")

# https://blog.sentry.io/logging-in-python-a-developers-guide/

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import joblib

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking, Dropout
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall, TruePositives, TrueNegatives, FalsePositives, FalseNegatives, F1Score
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import load_model

import shap

In [ ]:
from helpers.pages import *
from helpers.constants import *
from helpers.questions import *
from helpers.utils import *
from helpers.machine_learning import *

In [ ]:
pd.set_option('display.max_columns', 500)

## Extracting columns from python file

In [ ]:
# Function to read the text file and extract column names
def extract_columns_from_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        
    # Initialize empty lists for storing columns
    df_to_test_cols = []
    df_to_train_cols = []

    # Loop through each line and extract columns
    for line in lines:
        if line.startswith('df_to_test_cols'):
            # Extract column names for df_to_test_cols
            df_to_test_cols = eval(line.split('=')[1].strip())
        elif line.startswith('df_to_train_cols'):
            # Extract column names for df_to_train_cols
            df_to_train_cols = eval(line.split('=')[1].strip())

    # Check if lists are identical
    if df_to_test_cols == df_to_train_cols:
        logging.info("Column names extracted successfully")
    else:
        logging.error("Column names extracted are not identical")
        raise ValueError("Column names extracted are not identical")
        
    return df_to_test_cols

## Latex model evaluation

In [ ]:
def create_latex_table(report_dict, model, variant):
    # Convert the dictionary to a DataFrame
    report_df = pd.DataFrame(report_dict).transpose()

    # Round the values for better readability
    report_df = report_df.round(2)
    report_df = report_df.map(lambda x: f'{x:.2f}')

    # Support should be integer
    report_df['support'] = report_df['support'].map(lambda x: f'{int(float(x)):,}')
    
    # Replace value of accuracy in columns precision, recall with empty string
    report_df.loc['accuracy', 'precision'] = ''
    report_df.loc['accuracy', 'recall'] = ''

    # Replace value of accuracy in column support with max value of support
    report_df.loc['accuracy', 'support'] = report_df['support'].max()

    # Convert the DataFrame to a LaTeX table
    latex_table = report_df.to_latex()

    print("\\begin{table}[h!]\n\\centering\n")
    print(latex_table)
    print(f"\\caption{{Klasifikačný report pre {model}}}\n\\label{{tab:classification_report_{variant}}}\n\\end{{table}}\n\n")

## Extracting whole table of metrics

In [ ]:
def all_metrics(report_out, variant):
    report = report_out.copy()

    # Get all tests
    all_tests = report[report["set"] == "test"]

    # Rename algorithms
    all_tests["algorithm"] = all_tests["algorithm"].replace("logistic_regression", "Logistická regresia")
    all_tests["algorithm"] = all_tests["algorithm"].replace("random_forest", "Random Forest")
    all_tests["algorithm"] = all_tests["algorithm"].replace("gradient_boosting", "Gradient Boosting")
    all_tests["algorithm"] = all_tests["algorithm"].replace("xgboost", "XGBoost")
    all_tests["algorithm"] = all_tests["algorithm"].replace("decision_tree", "Rozhodovací strom")
    all_tests["algorithm"] = all_tests["algorithm"].replace("linear_svm", "Linear SVM")
    all_tests["algorithm"] = all_tests["algorithm"].replace("rbf_svm", "RBF SVM")
    all_tests["algorithm"] = all_tests["algorithm"].replace("poly_svm", "Poly SVM")
    all_tests["algorithm"] = all_tests["algorithm"].replace("sigmoid_svm", "Sigmoid SVM")

    # Remove all balanced random forest models and balanced bagging models
    all_tests = all_tests[~all_tests["algorithm"].str.contains("balanced")]

    # In columns metric and algorithm, capitalize the first letter
    all_tests["metric"] = all_tests["metric"].str.capitalize()

    # Rename columns algorithm, metric, macro avg and weighted avg columns to slovak
    all_tests = all_tests.rename(columns={"algorithm": "Algoritmus", "metric": "Metrika", "macro avg": "Priemer", "weighted avg": "Vážený priemer"})

    # Pivot the table to macro avg and weighted avg for each metric
    evaluation_pivot = all_tests.pivot(index="Algoritmus", columns="Metrika", values=["Vážený priemer", "Priemer"])

    # Drop support
    evaluation_pivot = evaluation_pivot.drop(columns="Support", level=1)

    # Order the table by precision
    evaluation_pivot = evaluation_pivot.sort_values(by=("Vážený priemer", "Precision"), ascending=False)
    
    # Round to 2 decimal places
    evaluation_pivot = evaluation_pivot.round(2)
    evaluation_pivot = evaluation_pivot.map(lambda x: f'{x:.2f}')

    print("\\begin{table}[h!]\n\\centering\n")
    print(evaluation_pivot.to_latex())
    print(f"\\caption{{Metriky úspešnosti modelov strojového učenia na testovacom datasete}}\n\\label{{tab:all_models_{variant}}}\n\\end{{table}}")

    return evaluation_pivot

## Text and Voice

In [ ]:
# text_and_voice_run = "20240515_015525" # balanced_accuracy new
text_and_voice_run = "20240514_160028" # balanced_accuracy
# text_and_voice_run = "20240514_173005" # precision_weighted

In [ ]:
text_and_voice_base_path = "data\\12_PDU_Aggregations_and_Models\\"

In [ ]:
text_and_voice_report = pd.read_csv(text_and_voice_base_path + f"report\\{text_and_voice_run}.csv", sep=";")

In [ ]:
# Filter only set with value test and metric precision
text_and_voice_evaluation = text_and_voice_report[(text_and_voice_report["set"] == "test") & (text_and_voice_report["metric"] == "precision")].sort_values(by="weighted avg", ascending=False)
text_and_voice_evaluation

In [ ]:
all_metrics(text_and_voice_report, "text_and_voice")

In [ ]:
text_and_voice_best_model_1_name = text_and_voice_evaluation.iloc[0]["algorithm"]
print(text_and_voice_best_model_1_name)
text_and_voice_best_model_2_name = text_and_voice_evaluation.iloc[1]["algorithm"]
print(text_and_voice_best_model_2_name)

In [ ]:
text_and_voice_best_model_1 = joblib.load(text_and_voice_base_path + f"models\\{text_and_voice_best_model_1_name}\\{text_and_voice_run}.joblib")
print(type(text_and_voice_best_model_1))
text_and_voice_best_model_2 = joblib.load(text_and_voice_base_path + f"models\\{text_and_voice_best_model_2_name}\\{text_and_voice_run}.joblib")
print(type(text_and_voice_best_model_2))

In [ ]:
text_and_voice_selected_columns_path = text_and_voice_base_path + f"selected_columns\\{text_and_voice_run}.py"
text_and_voice_selected_columns = extract_columns_from_file(text_and_voice_selected_columns_path)

In [ ]:
text_and_voice_test_set = pd.read_csv(text_and_voice_base_path + f"datasets\\{text_and_voice_run}_test.csv", sep=",")
print(len(text_and_voice_test_set))
text_and_voice_train_set = pd.read_csv(text_and_voice_base_path + f"datasets\\{text_and_voice_run}_train.csv", sep=",")
print(len(text_and_voice_train_set))

In [ ]:
text_and_voice_test_set

In [ ]:
text_and_voice_test_set = text_and_voice_test_set[text_and_voice_selected_columns]
text_and_voice_train_set = text_and_voice_train_set[text_and_voice_selected_columns]

In [ ]:
text_and_voice_X_test = text_and_voice_test_set.drop(["indicator_fg"], axis=1).reset_index(drop=True)
text_and_voice_y_test = text_and_voice_test_set['indicator_fg'].astype(int).reset_index(drop=True)
text_and_voice_X_train = text_and_voice_train_set.drop(["indicator_fg"], axis=1).reset_index(drop=True)
text_and_voice_y_train = text_and_voice_train_set['indicator_fg'].astype(int).reset_index(drop=True)

In [ ]:
# Print report for test set and text_and_voice_best_model_1
text_and_voice_y_pred_1 = text_and_voice_best_model_1.predict(text_and_voice_X_test)
print("Predicting for test dataset:")
print(classification_report(text_and_voice_y_test, text_and_voice_y_pred_1))
calculate_shap(text_and_voice_best_model_1, text_and_voice_X_train, text_and_voice_X_test, tree=True, pos_class=True)

In [ ]:
# Print report for test set and text_and_voice_best_model_2
text_and_voice_y_pred_2 = text_and_voice_best_model_2.predict(text_and_voice_X_test)
print("Predicting for test dataset:")
print(classification_report(text_and_voice_y_test, text_and_voice_y_pred_2))
calculate_shap(text_and_voice_best_model_2, text_and_voice_X_train, text_and_voice_X_test)

In [ ]:
text_and_voice_report1_dict = classification_report(text_and_voice_y_test, text_and_voice_y_pred_1, output_dict=True)
text_and_voice_report2_dict = classification_report(text_and_voice_y_test, text_and_voice_y_pred_2, output_dict=True)

In [ ]:
create_latex_table(text_and_voice_report1_dict, "algoritmus Rozhodovací strom.", "text_and_voice_1")

In [ ]:
create_latex_table(text_and_voice_report2_dict, "algoritmus Logistická regresia.", "text_and_voice_2")

## Mouse Metrics

In [ ]:
# mouse_metrics_run = "20240515_015532" # balanced_accuracy new
mouse_metrics_run = "20240514_160051" # balanced_accuracy
# mouse_metrics_run = "20240514_173010" # precision_weighted

In [ ]:
mouse_metrics_base_path = "data\\14_Mouse_Model\\"

In [ ]:
mouse_metrics_report = pd.read_csv(mouse_metrics_base_path + f"report\\{mouse_metrics_run}.csv", sep=";")

In [ ]:
# Filter only set with value test and metric precision
mouse_metrics_evaluation = mouse_metrics_report[(mouse_metrics_report["set"] == "test") & (mouse_metrics_report["metric"] == "precision")].sort_values(by="weighted avg", ascending=False)
mouse_metrics_evaluation

In [ ]:
all_metrics(mouse_metrics_report, "mouse_metrics")

In [ ]:
mouse_metrics_best_model_1_name = mouse_metrics_evaluation.iloc[0]["algorithm"]
print(mouse_metrics_best_model_1_name)
mouse_metrics_best_model_2_name = mouse_metrics_evaluation.iloc[1]["algorithm"]
print(mouse_metrics_best_model_2_name)

In [ ]:
mouse_metrics_best_model_1 = joblib.load(mouse_metrics_base_path + f"models\\{mouse_metrics_best_model_1_name}\\{mouse_metrics_run}.joblib")
print(type(mouse_metrics_best_model_1))
mouse_metrics_best_model_2 = joblib.load(mouse_metrics_base_path + f"models\\{mouse_metrics_best_model_2_name}\\{mouse_metrics_run}.joblib")
print(type(mouse_metrics_best_model_2))

In [ ]:
mouse_metrics_selected_columns_path = mouse_metrics_base_path + f"selected_columns\\{mouse_metrics_run}.py"
mouse_metrics_selected_columns = extract_columns_from_file(mouse_metrics_selected_columns_path)

In [ ]:
mouse_metrics_test_set = pd.read_csv(mouse_metrics_base_path + f"datasets\\{mouse_metrics_run}_test.csv", sep=",")
print(len(mouse_metrics_test_set))
mouse_metrics_train_set = pd.read_csv(mouse_metrics_base_path + f"datasets\\{mouse_metrics_run}_train.csv", sep=",")
print(len(mouse_metrics_train_set))

In [ ]:
mouse_metrics_test_set

In [ ]:
mouse_metrics_test_set = mouse_metrics_test_set[mouse_metrics_selected_columns]
mouse_metrics_train_set = mouse_metrics_train_set[mouse_metrics_selected_columns]

In [ ]:
mouse_metrics_X_test = mouse_metrics_test_set.drop(["indicator_fg"], axis=1).reset_index(drop=True)
mouse_metrics_y_test = mouse_metrics_test_set['indicator_fg'].astype(int).reset_index(drop=True)
mouse_metrics_X_train = mouse_metrics_train_set.drop(["indicator_fg"], axis=1).reset_index(drop=True)
mouse_metrics_y_train = mouse_metrics_train_set['indicator_fg'].astype(int).reset_index(drop=True)

In [ ]:
# Print report for test set and mouse_metrics_best_model_1
mouse_metrics_y_pred_1 = mouse_metrics_best_model_1.predict(mouse_metrics_X_test)
print("Predicting for test dataset:")
print(classification_report(mouse_metrics_y_test, mouse_metrics_y_pred_1))
calculate_shap(mouse_metrics_best_model_1, mouse_metrics_X_train, mouse_metrics_X_test, tree=True)

In [ ]:
# Print report for test set and mouse_metrics_best_model_2
mouse_metrics_y_pred_2 = mouse_metrics_best_model_2.predict(mouse_metrics_X_test)
print("Predicting for test dataset:")
print(classification_report(mouse_metrics_y_test, mouse_metrics_y_pred_2))
calculate_shap(mouse_metrics_best_model_2, mouse_metrics_X_train, mouse_metrics_X_test)

In [ ]:
mouse_metrics_report1_dict = classification_report(mouse_metrics_y_test, mouse_metrics_y_pred_1, output_dict=True)
mouse_metrics_report2_dict = classification_report(mouse_metrics_y_test, mouse_metrics_y_pred_2, output_dict=True)

In [ ]:
create_latex_table(mouse_metrics_report1_dict, "algoritmus Gradient Boosting.", "mouse_metrics_1")

In [ ]:
create_latex_table(mouse_metrics_report2_dict, "algoritmus Linear SVM.", "mouse_metrics_2")

## Mouse Trajectory

In [ ]:
# mouse_trajectory_run_1 = "generous-lion-25" # 50 0.2 50 0.2 https://wandb.ai/xsmrecek/mouse-movement-lie-detection/runs/vzcb9yp2?nw=nwuserxsmrecek
# mouse_trajectory_run_2 = "honest-dragon-30" # 128 0.2 https://wandb.ai/xsmrecek/mouse-movement-lie-detection/runs/aoxt8lgk?nw=nwuserxsmrecek
mouse_trajectory_run_1_raw = "earnest-plasma-32" # 50 0.2 50 0.2 https://wandb.ai/xsmrecek/mouse-movement-lie-detection/runs/xjyngoa9?nw=nwuserxsmrecek
mouse_trajectory_run_2_raw = "sleek-shape-31" # 128 0.2 https://wandb.ai/xsmrecek/mouse-movement-lie-detection/runs/itqpg34n?nw=nwuserxsmrecek

In [ ]:
mouse_trajectory_base_path = "data\\15_Neural_Net_Model\\"

In [ ]:
mouse_trajectory_model_1 = load_model(mouse_trajectory_base_path + f"models\\{mouse_trajectory_run_1_raw}.keras")
print(type(mouse_trajectory_run_1_raw))
mouse_trajectory_model_2 = load_model(mouse_trajectory_base_path + f"models\\{mouse_trajectory_run_2_raw}.keras")
print(type(mouse_trajectory_run_2_raw))

In [ ]:
mouse_trajectory_test_set = pd.read_csv(mouse_trajectory_base_path + f"data\\test_df.csv", sep=",")
len(mouse_trajectory_test_set)

In [ ]:
mouse_trajectory_test_set

In [ ]:
def prepare_sequences(new_df):
    df = new_df.groupby(['variant', 'respondent', 'page_name'])

    sequences = []
    labels = []

    for _, group in df:
        # Here, each group will be a DataFrame containing the rows for a specific observation
        sequences.append(group[['delta_x', 'delta_y']].values)
        labels.append(group['indicator_fg'].iloc[0])  # Assuming all values in indicator_fg are the same within a group

    # Convert lists to arrays for processing
    X = np.array(sequences, dtype=object)  # Keeping as an object array to handle variable lengths
    y = np.array(labels, dtype=float)

    return X, y

In [ ]:
def predict_new_sequences(model, sequences):
    predictions = []
    for sequence in sequences:
        # Since the model expects a batch dimension and potentially padding, adjust sequence shape
        sequence = np.array(sequence)[np.newaxis, :]  # Add batch dimension
        prediction = model.predict(sequence)
        predictions.append(prediction.flatten()[0])  # Flatten to get a single prediction value
    return predictions

In [ ]:
def process_and_predict(new_df, model):
    sequences, true_labels = prepare_sequences(new_df)
    predictions = predict_new_sequences(model, sequences)
    threshold = 0.5
    predicted_labels = [1 if p >= threshold else 0 for p in predictions]
    return predicted_labels, true_labels

In [ ]:
# Print report for test set and mouse_trajectory_model_1
predicted_labels_1, true_labels_1 = process_and_predict(mouse_trajectory_test_set, mouse_trajectory_model_1)
print("Predicting for test dataset:")
print(classification_report(true_labels_1, predicted_labels_1))

In [ ]:
# Print report for test set and mouse_trajectory_model_2
predicted_labels_2, true_labels_2 = process_and_predict(mouse_trajectory_test_set, mouse_trajectory_model_2)
print("Predicting for test dataset:")
print(classification_report(true_labels_2, predicted_labels_2))

In [ ]:
mouse_trajectory_report1_dict = classification_report(true_labels_1, predicted_labels_1, output_dict=True)
mouse_trajectory_report2_dict = classification_report(true_labels_2, predicted_labels_2, output_dict=True)

In [ ]:
create_latex_table(mouse_trajectory_report1_dict, "neurónovú sieť s dvomi LSTM vrstvami a dvomi dropout vrstvami.", "mouse_trajectory_1")

In [ ]:
create_latex_table(mouse_trajectory_report2_dict, "neurónovú sieť s jednou LSTM vrstvou a jednou dropout vrstvou.", "mouse_trajectory_2")